<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/pytorch/blob/main/feedforward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision as vision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [2]:


#Equivalent to
#flatten=transforms.Lambda(lambda x:torch.flatten(x,start_dim=0))

In [3]:
flatten=transforms.Lambda(lambda x:torch.flatten(x))
#since the input has 3 channels we need 3 values for mean and 3 for stdev
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),flatten])
#mnist_train=vision.datasets.MNIST(".",download=True,train=True,transform=transform)
#mnist_test=vision.datasets.MNIST(".",download=True,train=False,transform=transform)
#train_loader=torch.utils.data.DataLoader(mnist_train,batch_size=512,shuffle=True)
#test_loader=torch.utils.data.DataLoader(mnist_test,batch_size=512,shuffle=True)
cifar10_train=vision.datasets.CIFAR10(".",download=True,train=True,transform=transform)
cifar10_test=vision.datasets.CIFAR10(".",download=True,train=False,transform=transform)
train_loader=torch.utils.data.DataLoader(cifar10_train,batch_size=64,shuffle=True)
test_loader=torch.utils.data.DataLoader(cifar10_test,batch_size=64,shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


In [4]:
input_size=32*32*3
output_size=10
hidden_size1=128
hidden_size2=64
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.layer1=nn.Linear(input_size,hidden_size1)
    self.layer2=nn.Linear(hidden_size1,hidden_size2)
    self.layer3=nn.Linear(hidden_size2,output_size)
    self.relu=nn.Sigmoid()
  def forward(self,x):
    x=self.layer1(x)
    x=self.relu(x)
    x=self.layer2(x)
    x=self.relu(x)
    x=self.layer3(x)

    return x

model=Net().cuda()


In [8]:
import torch.optim as optim
epochs=10
optimizer=optim.SGD(model.parameters(),lr=0.01)
loss_fn=nn.CrossEntropyLoss()
running_loss=0.0
for epoch in range(epochs):
  for i,data in enumerate(train_loader):
    optimizer.zero_grad()
    img,label=data
    img,label=img.cuda(),label.cuda()
  
    output=model(img)
    loss=loss_fn(output,label)
  
    loss.backward()
    optimizer.step()
    running_loss=0.99*running_loss+0.01*loss.item()
  print(running_loss)
#  img=torch.flatten(img,start_dim=1)


1.9041827024799893
1.901585853311849
1.881568329190928
1.8768647609872688
1.868468963507351
1.8508025253272589
1.830192177356067
1.8310783336315406
1.8245061584497302
1.8106016664797309


In [9]:
total=0
correct=0
for data in test_loader:
  imgs,labels=data
  imgs=imgs.cuda()
  labels=labels.cuda()
  outputs=model(imgs)
  # the second return value is the index of the max i.e. argmax
  _,predicted=torch.max(outputs.data,1)
  total+=labels.size(0)
  correct+=(predicted==labels).sum()


In [10]:
print((correct/total).item())

0.35009998083114624
